In [ ]:
!pip3 install datasets==2.9.0 transformers==4.26.1 seqeval

In [ ]:
!pip install flax jax==0.4.2 jaxlib==0.4.2+cuda11.cudnn86 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

In [ ]:
# !python3 convert_t5x_checkpoint_to_flax.py \
#     --t5x_checkpoint_path out/chemprot/biot5x_base_text/checkpoint_1204500 \
#     --config_name t5_base_config.json \
#     --flax_dump_folder_path out/chemprot/biot5_flax_text


# from transformers import AutoModel, T5TokenizerFast

# model = AutoModel.from_pretrained("out/chemprot/biot5_flax_text", from_flax=True)
# tokenizer = T5TokenizerFast.from_pretrained("t5-base")

# model.save_pretrained("trl/out/chemprot_hf/biot5_pytorch_text")
# tokenizer.save_pretrained("trl/out/chemprot_hf/biot5_pytorch_text")

In [ ]:
from transformers import T5TokenizerFast, AutoModel

model = AutoModel.from_pretrained("trl/out/chemprot_hf/biot5_pytorch_text")
tokenizer = T5TokenizerFast.from_pretrained("trl/out/chemprot_hf/biot5_pytorch_text")


model.push_to_hub("justinphan3110/biot5_chemprot")
tokenizer.push_to_hub("justinphan3110/biot5_chemprot")

In [ ]:
import os

# os.environ['MODEL']='SciFive-base-Pubmed_PMC'
# os.environ['MODEL_PATH']=f"razent/{os.environ['MODEL']}"

os.environ['MODEL']='biot5x_base_hf'
os.environ['MODEL_PATH']=f"out/{os.environ['MODEL']}"

os.environ['task']='chemprot'
os.environ['datadir']=f"biot5x/data/{os.environ['task']}"
os.environ['outdir']=f"out/tmp/{os.environ['task']}_hf/{os.environ['MODEL']}"
os.environ["WANDB_DISABLED"] = "true"
  # --save_strategy epoch --evaluation_strategy epoch --load_best_model_at_end --metric_for_best_model F1  \

!python3 -u run_scifive_hf.py --model_name_or_path $MODEL_PATH \
  --train_file $datadir/train_blurb.tsv --validation_file $datadir/test_blurb.tsv --test_file $datadir/test_blurb.tsv \
  --do_train --do_predict --metric_name PRF1 --predict_with_generate \
  --per_device_train_batch_size 128 --per_device_eval_batch_size 128 --fp16_full_eval --gradient_accumulation_steps 1 --fp16 \
  --learning_rate 0.0005 --num_train_epochs 40 --max_source_length 256 --max_target_length 4 \
  --log_level warning --output_dir $outdir --overwrite_output_dir --from_flax

In [ ]:
# out/chemprot_hf/biot5_pytorch_text

import os

# os.environ['MODEL']='biot5_pytorch_text'
# os.environ['MODEL_PATH']=f"trl/out/chemprot_hf/{os.environ['MODEL']}"


os.environ['MODEL']='biot5_chemprot'
os.environ['MODEL_PATH']=f"justinphan3110/{os.environ['MODEL']}"
os.environ['task']='chemprot'
os.environ['datadir']=f"biot5x/data/{os.environ['task']}"
os.environ['outdir']=f"tmp/{os.environ['task']}_hf/{os.environ['MODEL']}"
os.environ["WANDB_DISABLED"] = "true"
  # --save_strategy epoch --evaluation_strategy epoch --load_best_model_at_end --metric_for_best_model F1  \

!python3 -u run_scifive_hf.py --model_name_or_path $MODEL_PATH \
  --train_file $datadir/train_blurb.tsv --validation_file $datadir/test_blurb.tsv --test_file $datadir/test_blurb_text.tsv \
  --do_predict --metric_name PRF1 --predict_with_generate \
  --per_device_train_batch_size 128 --per_device_eval_batch_size 128 --fp16_full_eval --gradient_accumulation_steps 1 --fp16 \
  --learning_rate 0.00005 --num_train_epochs 20 --max_source_length 256 --max_target_length 5 \
  --log_level warning --output_dir $outdir --overwrite_output_dir

In [ ]:
import os

os.environ['MODEL']='checkpoint_2'
os.environ['MODEL_PATH']=f"trl/out/test_trl_biot5_chemprot/{os.environ['MODEL']}"
os.environ['task']='chemprot'
os.environ['datadir']=f"biot5x/data/{os.environ['task']}"
os.environ['outdir']=f"tmp/{os.environ['task']}_hf/{os.environ['MODEL']}"
os.environ["WANDB_DISABLED"] = "true"
  # --save_strategy epoch --evaluation_strategy epoch --load_best_model_at_end --metric_for_best_model F1  \

!python3 -u run_scifive_hf.py --model_name_or_path $MODEL_PATH \
  --train_file $datadir/train_blurb.tsv --validation_file $datadir/test_blurb.tsv --test_file $datadir/test_blurb_text.tsv \
  --do_predict --metric_name PRF1 --predict_with_generate \
  --per_device_train_batch_size 512 --per_device_eval_batch_size 512 --fp16_full_eval --gradient_accumulation_steps 1 --fp16 \
  --learning_rate 0.00005 --num_train_epochs 20 --max_source_length 256 --max_target_length 5 \
  --log_level warning --output_dir $outdir --overwrite_output_dir

In [ ]:
import os

os.environ['MODEL']='biot5x'
os.environ['MODEL_PATH']=f"out/ddi_hf/{os.environ['MODEL']}"
os.environ['task']='ddi'
os.environ['datadir']=f"biot5x/data/{os.environ['task']}"
os.environ['outdir']=f"out/{os.environ['task']}_hf/{os.environ['MODEL']}"
os.environ["WANDB_DISABLED"] = "true"
  # --save_strategy epoch --evaluation_strategy epoch --load_best_model_at_end --metric_for_best_model F1  \

!python3 -u run_scifive_hf.py --model_name_or_path $MODEL_PATH \
  --train_file $datadir/train_blurb.tsv --validation_file $datadir/test_blurb.tsv --test_file $datadir/test_blurb.tsv \
  --do_predict --metric_name PRF1 --predict_with_generate \
  --per_device_train_batch_size 128 --per_device_eval_batch_size 128 --fp16_full_eval --gradient_accumulation_steps 1 --fp16 \
  --learning_rate 0.00005 --num_train_epochs 20 --max_source_length 256 --max_target_length 4 \
  --log_level warning --output_dir $outdir --overwrite_output_dir --from_flax